# Download Latest JPL Ephemerides as Numpy Data

Most recent planetary ephemeris is de438; uploaded on 28-Aug-2019.<br>
See https://pypi.org/project/jplephem/ for jplephem tutorial

In [1]:
import numpy as np
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, BarycentricMeanEcliptic
from jplephem.spk import SPK

<b>Load the de438 kernel</b>

In [2]:
kernel = SPK.open('data/jpl/ephemeris/de438.bsp')

In [3]:
print(kernel)

File type DAF/SPK and format LTL-IEEE with 14 segments:
2287184.50..2688976.50  Solar System Barycenter (0) -> Mercury Barycenter (1)
2287184.50..2688976.50  Solar System Barycenter (0) -> Venus Barycenter (2)
2287184.50..2688976.50  Solar System Barycenter (0) -> Earth Barycenter (3)
2287184.50..2688976.50  Solar System Barycenter (0) -> Mars Barycenter (4)
2287184.50..2688976.50  Solar System Barycenter (0) -> Jupiter Barycenter (5)
2287184.50..2688976.50  Solar System Barycenter (0) -> Saturn Barycenter (6)
2287184.50..2688976.50  Solar System Barycenter (0) -> Uranus Barycenter (7)
2287184.50..2688976.50  Solar System Barycenter (0) -> Neptune Barycenter (8)
2287184.50..2688976.50  Solar System Barycenter (0) -> Pluto Barycenter (9)
2287184.50..2688976.50  Solar System Barycenter (0) -> Sun (10)
2287184.50..2688976.50  Earth Barycenter (3) -> Moon (301)
2287184.50..2688976.50  Earth Barycenter (3) -> Earth (399)
2287184.50..2688976.50  Mercury Barycenter (1) -> Mercury (199)
228718

<b>Select a date range</b><br>
Most recent asteroid integration as of MJD 59,000.  One century in either direction would span 22,475 to 95,525.

In [4]:
mjd_int = 59000
mjd_half_width = 20*365.25
mjd0 = mjd_int - mjd_half_width
mjd1 = mjd_int + mjd_half_width
jd0 = mjd0 + 2400000.5
jd1 = mjd1 + 2400000.5
jd = np.arange(jd0, jd1+1.0)

In [5]:
jd

array([2451695.5, 2451696.5, 2451697.5, ..., 2466303.5, 2466304.5,
       2466305.5])

<b>Conversion Between AU and kilometers</b>

In [6]:
au2km = 149597870.7

In [7]:
km2au = 1.0 / au2km

<b>Extract position and velocity for Sun \& Planet Barycenters</b>

In [8]:
q_sun_km, v_sun_km = kernel[0,10].compute_and_differentiate(jd)
q_mercury_km, v_mercury_km = kernel[0,1].compute_and_differentiate(jd)
q_venus_km, v_venus_km = kernel[0,2].compute_and_differentiate(jd)
q_emb_km, v_emb_km = kernel[0,3].compute_and_differentiate(jd)
q_mars_km, v_mars_km = kernel[0,4].compute_and_differentiate(jd)
q_jupiter_km, v_jupiter_km = kernel[0,5].compute_and_differentiate(jd)
q_saturn_km, v_saturn_km = kernel[0,6].compute_and_differentiate(jd)
q_uranus_km, v_uranus_km = kernel[0,7].compute_and_differentiate(jd)
q_neptune_km, v_neptune_km = kernel[0,8].compute_and_differentiate(jd)
q_pluto_km, v_pluto_km = kernel[0,9].compute_and_differentiate(jd)

<b>Extract position and velocity for Earth and Moon from Earth-Moon Barycenter</b>

In [9]:
q_emb_earth_km, v_emb_earth_km = kernel[3,399].compute_and_differentiate(jd)
q_emb_moon_km, v_emb_moon_km = kernel[3,301].compute_and_differentiate(jd)

In [10]:
# Earth and Moon from EMB
q_earth_km = q_emb_km + q_emb_earth_km
v_earth_km = v_emb_km + v_emb_earth_km
q_moon_km = q_emb_km + q_emb_moon_km
v_moon_km = v_emb_km + v_emb_moon_km

<b>Concatenate position and velocity</b>

In [11]:
qv_sun_km = np.vstack([q_sun_km, v_sun_km])
qv_mercury_km = np.vstack([q_mercury_km, v_mercury_km])
qv_venus_km = np.vstack([q_venus_km, v_venus_km])
qv_emb_km = np.vstack([q_emb_km, v_emb_km])
qv_mars_km = np.vstack([q_mars_km, v_mars_km])
qv_jupiter_km = np.vstack([q_jupiter_km, v_jupiter_km])
qv_saturn_km = np.vstack([q_saturn_km, v_saturn_km])
qv_uranus_km = np.vstack([q_uranus_km, v_uranus_km])
qv_neptune_km = np.vstack([q_neptune_km, v_neptune_km])
qv_pluto_km = np.vstack([q_pluto_km, v_pluto_km])

# Earth and Moon from EMB
qv_earth_km = np.vstack([q_earth_km, v_earth_km])
qv_moon_km = np.vstack([q_moon_km, v_moon_km])

<b>Position and Velocity in AU in the ICRS</b>

In [12]:
qv_sun_icrs = qv_sun_km * km2au
qv_mercury_icrs = qv_mercury_km * km2au
qv_venus_icrs = qv_venus_km * km2au
qv_emb_icrs = qv_emb_km * km2au
qv_mars_icrs = qv_mars_km * km2au
qv_jupiter_icrs = qv_jupiter_km * km2au
qv_saturn_icrs = qv_saturn_km * km2au
qv_uranus_icrs = qv_uranus_km * km2au
qv_neptune_icrs = qv_neptune_km * km2au
qv_pluto_icrs = qv_pluto_km * km2au

# Earth and Moon from EMB
qv_earth_icrs = qv_earth_km * km2au
qv_moon_icrs = qv_moon_km * km2au

<b> Position and Velocity in the Barycentric Mean Ecliptic Frame>/b>

In [13]:
def icrs2bme(qv_icrs):
    """Convert a 6xN array [q_body, v_body] from ICRS to BME frame"""
    # Build coordinate object with position in AU and velocity in AU / day
    body_icrs = SkyCoord(x=qv_icrs[0]*au, y=qv_icrs[1]*au, z=qv_icrs[2]*au, 
                         v_x=qv_icrs[3]*au/day, v_y=qv_icrs[4]*au/day, v_z=qv_icrs[5]*au/day,
                         frame=ICRS, representation_type='cartesian',  differential_type='cartesian')
    # Extract position in BME with units of AU
    q = body_icrs.barycentricmeanecliptic.cartesian.xyz.to(au).value
    # Extract velocity in BME; convert units to AU / day
    v = body_icrs.barycentricmeanecliptic.velocity.d_xyz.to(au / day).value
    # Stack position and velocity into one 6xN array in the BME frame
    qv = np.vstack([q, v])
    return qv

In [14]:
qv_sun = icrs2bme(qv_sun_icrs)
qv_mercury = icrs2bme(qv_mercury_icrs)
qv_venus = icrs2bme(qv_venus_icrs)
qv_emb = icrs2bme(qv_emb_icrs)
qv_mars = icrs2bme(qv_mars_icrs)
qv_jupiter = icrs2bme(qv_jupiter_icrs)
qv_saturn = icrs2bme(qv_saturn_icrs)
qv_uranus = icrs2bme(qv_uranus_icrs)
qv_neptune = icrs2bme(qv_neptune_icrs)
qv_pluto = icrs2bme(qv_pluto_icrs)

qv_earth = icrs2bme(qv_earth_icrs)
qv_moon = icrs2bme(qv_moon_icrs)

In [15]:
emb_icrs = SkyCoord(x=qv_emb_icrs[0]*au, y=qv_emb_icrs[1]*au, z=qv_emb_icrs[2]*au, 
                    v_x=qv_emb_icrs[3]*au/day, v_y=qv_emb_icrs[4]*au/day, v_z=qv_emb_icrs[5]*au/day,
                    frame=ICRS, representation_type='cartesian',  differential_type='cartesian')

In [16]:
earth_icrs = SkyCoord(x=qv_earth_icrs[0]*au, y=qv_earth_icrs[1]*au, z=qv_earth_icrs[2]*au, 
                      v_x=qv_earth_icrs[3]*au/day, v_y=qv_earth_icrs[4]*au/day, v_z=qv_earth_icrs[5]*au/day,
                      frame=ICRS, representation_type='cartesian',  differential_type='cartesian')

In [17]:
q_emb = emb_icrs.barycentricmeanecliptic.cartesian.xyz.value

In [18]:
v_emb = emb_icrs.barycentricmeanecliptic.velocity.d_xyz.to(au / day).value

In [19]:
qv_emb = np.vstack([q_emb, v_emb])

<b>Sanity Check: Do we Recover Mean perihelion and aphelion ~ 1.0 AU?</b>

In [20]:
perihelion = np.min(np.linalg.norm(qv_earth[0:3] - qv_sun[0:3], axis=0))
aphelion = np.max(np.linalg.norm(qv_earth[0:3] - qv_sun[0:3], axis=0))
au_recovered = (perihelion + aphelion) / 2.0

In [21]:
print(f'perihelion   = {perihelion}')
print(f'aphelion     = {aphelion}')
print(f'recovered AU = {au_recovered}')

perihelion   = 0.9832438619468017
aphelion     = 1.0167543280740092
recovered AU = 0.9999990950104054


<b>Sanity Check: Do we Match First Row of Horizon Download for Earth-Moon Barycenter?</b>

In [22]:
2451700.5 - jd[0]

5.0

In [23]:
jd[5]

2451700.5

In [24]:
q_emb_jpl = qv_emb[0:3, 5]
q_sun_jpl = qv_sun[0:3, 5]
q_sun_emb_jpl = q_emb_jpl - q_sun_jpl

In [25]:
v_emb_jpl = qv_emb[3:6, 5]
v_sun_jpl = qv_sun[3:6, 5]
v_sun_emb_jpl = v_emb_jpl - v_sun_jpl

In [26]:
q_emb_hrzn = np.array([-2.748856606750498E-01, -9.822784617285307E-01, 1.904193980135841E-04])
q_sun_hrzn = np.array([-6.196323345882849E-03, -3.853552818516107E-03, 1.881871367065105E-04])
q_sun_emb_hrzn = q_emb_hrzn - q_sun_hrzn

In [27]:
v_emb_hrzn = np.array([1.631683037292223E-02, -4.626739131144278E-03, -1.411710983045592E-07])
v_sun_hrzn = np.array([6.728156861636702E-06, -6.135508083871250E-06, -1.381672089819167E-07])
v_sun_emb_hrzn = v_emb_hrzn - v_sun_hrzn

*Difference in Earth-Moon Barycenter absolute position*

In [28]:
q_emb_diff = q_emb_jpl - q_emb_hrzn
q_emb_diff

array([ 3.07421371e-08, -8.12610756e-09, -1.40250597e-07])

In [29]:
np.linalg.norm(q_emb_diff)

1.4381009203474344e-07

In [31]:
v_emb_diff = v_emb_jpl - v_emb_hrzn
v_emb_diff

array([ 1.48886601e-10,  5.28031057e-10, -2.45935938e-09])

In [38]:
# np.linalg.norm(v_emb_diff)

In [33]:
np.linalg.norm(v_emb_diff) / np.linalg.norm(v_emb_hrzn)

1.4857253671305304e-07

*Difference in Earth-Moon Barycenter relative position vs. Sun*

In [34]:
q_sun_emb_diff = q_sun_emb_jpl - q_sun_emb_hrzn
q_sun_emb_diff

array([ 3.16746421e-08, -8.69851491e-09, -1.40085992e-07])

In [36]:
np.linalg.norm(q_sun_emb_diff)

1.4388548300229448e-07

In [37]:
v_sun_emb_diff = v_sun_emb_jpl - v_sun_emb_hrzn
v_sun_emb_diff

array([ 1.49572275e-10,  5.28025890e-10, -2.45765275e-09])

In [40]:
np.linalg.norm(v_sun_emb_diff) / np.linalg.norm(v_sun_emb_hrzn)

1.4854800848565443e-07